# 평가 구성하기
특정 작업에 대해 Claude가 최대한 높은 정확도를 보이도록 최적화하는 과정은 경험적 과학이자 지속적인 개선의 여정입니다. 프롬프트 변경이 주요 지표에서 모델 성능을 향상했는지 확인하거나, 모델이 프로덕션 환경에 배포될 만큼 충분한 성능인지 측정하려는 경우, 효과적인 오프라인 평가 시스템 구축은 성공에 매우 중요합니다.

이 가이드에서는 평가를 구성하는 일반적인 패턴과 이때 따를 만한 유용한 경험적 규칙들을 살펴보겠습니다.

## 평가의 구성 요소
평가는 일반적으로 다음 네 가지 요소로 구성됩니다.
- 모델에 제공되는 입력 프롬프트: Claude에게 이 프롬프트를 기반으로 응답을 생성하도록 요청합니다. 평가 설계 시, 입력 열에는 테스트 시점에 프롬프트 템플릿에 주입될 다양한 변수 입력값들이 포함되는 경우가 많습니다.
- 모델 출력: 평가 대상 모델에 입력 프롬프트를 실행하여 얻은 결과입니다.
- 기준 답변 ("Golden Answer"): 모델 출력을 비교하기 위한 정답입니다. 기준 답변은 반드시 정확히 일치해야 하는 값일 수도 있고, 채점자가 점수를 부여할 때 참고할 수 있는 이상적인 답변의 예시일 수도 있습니다.
- 점수: 아래에서 설명할 채점 방법 중 하나를 사용하여 생성되며, 해당 질문에 대한 모델의 성능을 나타냅니다.

## 평가 채점 방법
평가 과정에서 시간과 비용이 많이 소요될 수 있는 두 가지 주요 요소가 있습니다. 첫째는 평가용 질문과 기준 답변을 작성하는 것이고, 둘째는 채점입니다. 사용 가능한 데이터 세트가 없거나, 수동으로 질문을 만들지 않고는 데이터 세트를 구축할 방법이 없는 경우, 질문과 기준 답변 작성에 상당한 시간이 소요될 수 있습니다 (이때 Claude를 활용하여 질문 생성을 고려해볼 수 있습니다!). 하지만 이는 일반적으로 일회성 고정 비용이라는 장점이 있습니다. 한번 질문과 기준 답변을 작성해두면, 이를 다시 작성해야 하는 경우는 드뭅니다. 반면, 채점은 평가를 다시 실행할 때마다 지속적으로 발생하는 비용이며, 평가를 자주 반복 실행할 가능성이 높습니다. 따라서 빠르고 비용 효율적으로 채점 가능한 평가 시스템을 구축하는 것이 평가 설계의 핵심 고려 사항이 되어야 합니다.

평가를 채점하는 일반적인 방법은 다음과 같이 세 가지가 있습니다.
- **코드 기반 채점:** 표준 코드(주로 문자열 일치 및 정규 표현식 사용)를 통해 모델의 출력을 채점합니다. 일반적인 방식으로는 답변과 정확히 일치하는지 확인하거나, 문자열 내에 특정 핵심 구문이 포함되어 있는지 검사합니다. 이러한 채점 방식을 적용할 수 있도록 평가를 설계할 수 있다면, 이는 속도와 신뢰성 면에서 단연 최고의 방법입니다. 하지만 모든 평가에 이 방식을 적용할 수 있는 것은 아닙니다.
- **수동 채점 (Human Grading):** 평가자가 직접 모델이 생성한 답변을 기준 답변과 비교하여 점수를 매깁니다. 이 방법은 거의 모든 유형의 작업에 적용 가능하여 가장 폭넓게 사용될 수 있는 채점 방식이지만, 특히 대규모 평가의 경우 시간과 비용이 매우 많이 소요됩니다. 가능하다면 수동 채점이 필요한 평가 설계는 피하는 것이 좋습니다.
- **모델 기반 채점:** Claude는 스스로의 답변을 평가하는 데에도 뛰어난 능력을 보이며, 과거에는 사람의 판단이 필요했던 창의적 글쓰기의 어조 분석이나 자유 형식 질문에 대한 답변의 정확성 평가 등 다양한 작업의 채점에 활용될 수 있습니다. 이는 Claude를 위한 _채점자 프롬프트_를 작성함으로써 가능해집니다.

각 채점 방식의 예를 통해 자세히 살펴보겠습니다.

### 코드 기반 채점
여기서는 Claude에게 특정 동물이 가진 다리의 수를 정확히 식별하도록 요청하는 평가를 채점합니다. Claude가 다리의 개수만을 숫자로 출력하도록 유도하고, 정확한 값 일치를 기준으로 하는 코드 기반 채점기를 사용할 수 있도록 평가를 설계합니다.

In [ ]:
# 필요한 패키지 설치 및 읽기, anthropic 클라이언트 생성.
%pip install anthropic

In [2]:
from anthropic import Anthropic
client = Anthropic()
MODEL_NAME = "claude-3-opus-20240229"

In [6]:
# 이 작업의 입력 프롬프트 템플릿을 정의합니다.
def build_input_prompt(animal_statement):
    user_content = f"""제시된 동물에 대한 설명을 읽고, 해당 동물의 다리 개수를 판단하는 것이 당신의 임무입니다.
    
    동물에 대한 설명은 다음과 같습니다:
    <animal_statement>{animal_statement}</animal_statement>
    
    이 동물은 다리가 몇 개입니까? 다리의 개수를 정수형 숫자로만 반환하고, 다른 정보는 포함하지 마십시오."""

    messages = [{'role': 'user', 'content': user_content}]
    return messages

In [4]:
# 평가 데이터셋을 정의합니다 (실제로는 jsonl 또는 csv 파일 형태로 구성할 수 있습니다).
eval = [
    {
        "animal_statement": '이 동물은 인간입니다.',
        "golden_answer": '2'
    },
        {
        "animal_statement": '이 동물은 뱀입니다.',
        "golden_answer": '0'
    },
        {
        "animal_statement": '여우가 다리 한 개를 잃었지만, 마법처럼 잃었던 다리가 다시 생겨났고, 거기에 더해 신비한 추가 다리까지 하나 더 돋아났습니다.',
        "golden_answer": '5'
    }
]

In [7]:
# 각 입력에 대한 모델의 응답(완료)을 가져옵니다.
# get_completion 함수를 정의합니다 (앞서 설명한 중단 시퀀스 포함).
def get_completion(messages):
    response = client.messages.create(
        model=MODEL_NAME,
        max_tokens=5,
        messages=messages
    )
    return response.content[0].text

# 평가 데이터셋의 각 질문에 대한 모델 응답을 가져옵니다.
outputs = [get_completion(build_input_prompt(question['animal_statement'])) for question in eval]

# 생성된 출력들을 간략히 살펴보겠습니다.
for output, question in zip(outputs, eval):
    print(f"제시된 동물 설명: {question['animal_statement']}\n기준 답변: {question['golden_answer']}\n모델 출력: {output}\n")

제시된 동물 설명: 이 동물은 인간입니다.
기준 답변: 2
모델 출력: 2

제시된 동물 설명: 이 동물은 뱀입니다.
기준 답변: 0
모델 출력: 0

제시된 동물 설명: 여우가 다리 한 개를 잃었지만, 마법처럼 잃었던 다리가 다시 생겨났고, 거기에 더해 신비한 추가 다리까지 하나 더 돋아났습니다.
기준 답변: 5
모델 출력: 5



In [8]:
# 모델의 응답을 기준 답변과 비교하여 정확도를 확인합니다.
# 채점 함수를 정의합니다.
def grade_completion(output, golden_answer):
    return output == golden_answer

# 모델 출력들에 대해 채점 함수를 실행하고 전체 점수를 출력합니다.
grades = [grade_completion(output, question['golden_answer']) for output, question in zip(outputs, eval)]
print(f"채점 결과: {sum(grades)/len(grades)*100}%")

채점 결과: 100.0%


### 수동 채점 (Human Grading)
이제 Claude에게 일련의 개방형 질문을 하고 그 답변을 평가하는 상황을 가정해 보겠습니다. 이는 범용 채팅 어시스턴트 평가와 유사할 수 있습니다. 안타깝게도, 이러한 질문에 대한 답변은 매우 다양할 수 있어 코드로 채점하기 어렵습니다. 이 경우 활용할 수 있는 한 가지 방법이 바로 수동 채점입니다.

In [9]:
# 이 작업의 입력 프롬프트 템플릿을 정의합니다.
def build_input_prompt(question):
    user_content = f"""다음 질문에 답변해 주십시오:
    <question>{question}</question>"""

    messages = [{'role': 'user', 'content': user_content}]
    return messages

In [10]:
# 평가 데이터셋을 정의합니다. 이 작업의 경우, 수동 채점자에게 제공할 가장 효과적인 "기준 답변"은 모델 출력에서 확인해야 할 사항에 대한 상세한 지침입니다.
eval = [
    {
        "question": '오늘 할 운동 계획을 짜주세요. 다리 당기기 운동 최소 50회, 팔 당기기 운동 최소 50회, 그리고 코어 운동 10분을 포함해야 합니다.',
        "golden_answer": '정확한 답변은 다음 요소를 포함하는 운동 계획이어야 합니다: 50회 이상의 다리 당기기 운동 (예: 데드리프트. 스쿼트와 같은 밀기 운동은 해당되지 않음), 50회 이상의 팔 당기기 운동 (예: 로우. 프레스와 같은 밀기 운동은 해당되지 않음), 그리고 10분간의 코어 운동. 스트레칭이나 동적 워밍업은 포함될 수도 있고 안 될 수도 있지만, 그 외 다른 주요 운동은 포함되어서는 안 됩니다.'
    },
    {
        "question": '제인에게 수련회 때문에 오전 9시에 사무실 앞에서 만나자고 이메일을 보내주세요.',
        "golden_answer": '어시스턴트는 이메일 발송 기능이 없으므로, 정확한 답변은 이메일 발송 요청을 거절해야 합니다. 이메일 초안을 제안하는 것은 허용되지만, 실제로 이메일을 보내려고 시도하거나, 이메일 발송 함수를 호출하거나, 이메일 발송과 관련된 추가 정보(예: 수신자 이메일 주소)를 문의해서는 안 됩니다.'
    },
    {
        "question": '2024년 슈퍼볼 우승팀과 준우승팀은 누구인가요?', # Claude의 학습 데이터는 해당 시점 이후이므로 이 질문에 정확히 답변할 수 없을 것입니다.
        "golden_answer": '정답은 캔자스시티 치프스가 샌프란시스코 포티나이너스를 꺾었다고 명시합니다.'
    }
]

In [11]:
# 각 입력에 대한 모델의 응답을 가져옵니다.
# get_completion 함수를 정의합니다 (앞서 설명한 중단 시퀀스 포함).
def get_completion(messages):
    response = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2048,
        messages=messages
    )
    return response.content[0].text

# 평가 데이터셋의 각 질문에 대한 모델 응답을 가져옵니다.
outputs = [get_completion(build_input_prompt(question['question'])) for question in eval]

# 생성된 출력들을 간략히 살펴보겠습니다.
for output, question in zip(outputs, eval):
    print(f"질문 내용: {question['question']}\n기준 답변: {question['golden_answer']}\n모델 출력: {output}\n")

질문 내용: 오늘 할 운동 계획을 짜주세요. 다리 당기기 운동 최소 50회, 팔 당기기 운동 최소 50회, 그리고 코어 운동 10분을 포함해야 합니다.
기준 답변: 정확한 답변은 다음 요소를 포함하는 운동 계획이어야 합니다: 50회 이상의 다리 당기기 운동 (예: 데드리프트. 스쿼트와 같은 밀기 운동은 해당되지 않음), 50회 이상의 팔 당기기 운동 (예: 로우. 프레스와 같은 밀기 운동은 해당되지 않음), 그리고 10분간의 코어 운동. 스트레칭이나 동적 워밍업은 포함될 수도 있고 안 될 수도 있지만, 그 외 다른 주요 운동은 포함되어서는 안 됩니다.
모델 출력: 오늘을 위한 운동 계획을 아래와 같이 제안합니다. 이 계획에는 최소 50회의 다리 당기기 운동, 50회의 팔 당기기 운동, 그리고 10분의 코어 운동이 포함됩니다.

다리 당기기 운동:
1. 햄스트링 컬 (누워서 또는 앉아서): 3세트 x 12회 (총 36회)
2. 싱글 레그 루마니안 데드리프트: 각 다리당 2세트 x 10회 (총 40회)

팔 당기기 운동:
1. 벤트오버 로우: 3세트 x 10회 (총 30회)
2. 친업 또는 어시스티드 친업: 3세트 x 8회 (총 24회)

코어 운동 (10분):
1. 플랭크: 1분씩 3세트
2. 러시안 트위스트: 3세트 x 20회 (총 60회)
3. 바이시클 크런치: 3세트 x 20회 (총 60회)

워밍업: 본 운동 시작 전, 근육을 준비시키기 위해 5-10분간 가벼운 유산소 운동과 동적 스트레칭을 실시하세요.

휴식: 각 세트 사이에는 60-90초, 서로 다른 운동 사이에는 2-3분간 휴식합니다.

쿨다운: 운동 후 근육 회복을 돕고 유연성을 향상시키기 위해 5-10분간 정적 스트레칭으로 마무리합니다.

항상 몸의 소리에 귀 기울이고, 올바른 자세를 유지하며, 자신의 체력 수준에 맞게 무게, 저항, 또는 반복 횟수를 조절하는 것을 잊지 마십시오. 운동 중에는 수분을 충분히 섭취하고, 우려되는 점이 있거나 올바른 자세에 대한 지도가 필요하다면 주저하지 말고 피트

이러한 유형의 질문은 사람이 직접 채점해야 합니다. 따라서 지금부터는 모델의 출력과 기준 답변을 직접 비교하여 평가하거나, 혹은 출력과 기준 답변을 CSV 파일로 저장하여 다른 평가자에게 전달하여 채점을 진행할 수 있습니다.

### 모델 기반 채점
위에서 언급한 평가를 매번 수동으로 채점하는 것은, 특히 평가 규모가 수십, 수백, 혹은 수천 개의 질문으로 커질 경우 매우 번거롭고 시간이 많이 소요되는 작업이 될 것입니다. 다행히 더 효율적인 방법이 있습니다! 바로 Claude에게 채점을 맡기는 것입니다. 앞서 사용한 동일한 평가 데이터셋과 모델 응답을 활용하여 이 방법을 어떻게 적용할 수 있는지 살펴보겠습니다.

In [12]:
# 먼저 "채점자 프롬프트" 템플릿을 정의합니다.
def build_grader_prompt(answer, rubric):
    user_content = f"""당신은 어시스턴트가 특정 질문에 대해 생성한 답변과, 그 답변이 정답인지 오답인지를 판단하는 기준이 담긴 루브릭을 제공받게 됩니다.
    
    어시스턴트가 생성한 답변은 다음과 같습니다:
    <answer>{answer}</answer>
    
    답변의 정오답을 판단하는 루브릭은 다음과 같습니다:
    <rubric>{rubric}</rubric>
    
    답변이 루브릭의 기준을 모두 충족하면 '정답'이며, 그렇지 않으면 '오답'입니다.
    먼저, <thinking></thinking> 태그 안에서 루브릭을 바탕으로 답변이 정답인지 오답인지 판단하는 과정을 서술하십시오. 그 후, <correctness></correctness> 태그 안에 답변이 정답이면 'correct'를, 오답이면 'incorrect'를 출력하십시오."""

    messages = [{'role': 'user', 'content': user_content}]
    return messages

# 이제 전체 채점 함수 grade_completion을 정의합니다.
import re
def grade_completion(output, golden_answer):
    messages = build_grader_prompt(output, golden_answer)
    completion = get_completion(messages)
    # 모델 응답에서 최종 판단 레이블('correct' 또는 'incorrect')만 추출합니다 (사고 과정은 제외).
    pattern = r'<correctness>(.*?)</correctness>'
    match = re.search(pattern, completion, re.DOTALL)
    if match:
        return match.group(1).strip()
    else:
        raise ValueError("<correctness></correctness> 태그를 찾을 수 없습니다. 모델 응답 형식을 확인해주세요.")

# 모델 출력들에 대해 채점 함수를 실행하고 전체 점수를 출력합니다.
grades = [grade_completion(output, question['golden_answer']) for output, question in zip(outputs, eval)]
print(f"채점 결과: {grades.count('correct')/len(grades)*100}%")

채점 결과: 66.66666666666666%


보시는 바와 같이, Claude 기반 채점기는 Claude의 응답을 높은 정확도로 분석하고 채점할 수 있어, 평가에 소요되는 귀중한 시간을 절약해줍니다.

지금까지 평가를 위한 다양한 채점 설계 패턴에 대해 알아보았습니다. 이제 여러분만의 평가를 구성할 준비가 되셨을 것입니다. 시작에 도움이 될 만한 몇 가지 지침을 아래에 제시합니다.
- 가능한 경우, 특정 작업에 맞춤화된 평가를 구성하고, 평가 데이터셋 내 질문의 분포와 난이도가 실제 사용 사례의 분포와 유사하도록 노력하십시오.
- 모델 기반 채점 방식이 특정 작업에 적합한지 확인하는 유일한 방법은 직접 시도해보는 것입니다. 실제로 적용해보고 일부 샘플 결과를 검토하여 해당 작업에 적합한지 판단하십시오.
- 때로는 약간의 창의적인 설계만으로도 자동화 가능한 평가를 만들 수 있습니다. 작업의 본질을 유지하면서도 채점이 자동화될 수 있는 방식으로 질문을 구성해보십시오. 예를 들어, 질문을 객관식 형태로 변형하는 것이 일반적인 전략 중 하나입니다.
- 일반적으로, 소량의 고품질 질문보다는 대량의 (품질이 다소 낮더라도) 다양한 질문으로 구성된 평가가 더 유용할 수 있습니다.